In [1]:
from glob import glob #여러 파일의 경로를 가져오기위한 라이브러리이자 함수
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64

In [ ]:
load_dotenv()#.env파일에 저장된 환경변수를 불러오는 함수
api_key=os.getenv("OPENAI_API_KEY")
client=OpenAI(api_key=api_key)#api_key를 이용하여 OpenAI와 통신

def encode_image(image_path):
    with open(image_path, "rb") as image_file: #rb(바이너리모드): 0과1의 이진 데이터 그대로 읽기/ 이미지는 바이트로 구성되어있기때문
        return base64.b64encode(image_file.read()).decode("utf-8")
#이진 데이터를 base64형식(텍스트 형태)으로 인코딩 -> 인코딩된 데이터를 바이트에서 문자열로 변환

In [ ]:
def image_quiz(image_path):
    base64_image=encode_image(image_path)
    #GPT는 지시(promt)에  따라 행동하는 구조이므로 지시문 작성
    quize_prompt="""
    제공한 이미지를 바탕으로, 다음과 같은 양식으로 퀴즈를 만들어 주세요.
    정답은 1~4 중 하나만 해당하도록 출제하세요.
    아래는 예시입니다.
    ---예시---

    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
    1. 베이커리에서 사람들이 빵을 사는 모습이 담겨 있습니다.
    2. 맨 앞에 서 있는 사람은 빨간색 셔츠를 입었습니다.
    3. 기차를 타기 위해 줄을 서 있는 사람들이 있습니다.
    4. 점원은 노란색 티셔츠를 입었습니다.

    정답: 4. 점원은 노란색 티셔츠가 아닌 파란색 티셔츠를 입었습니다.
    (주의: 정답은 1~4 중 하나만 선택하도록 출제하세요.)
    ====
    """

    messages=[
        {
            "role": "user",
            "content":[
                {
                    "type": "text",
                    "text": quize_prompt
                },
                {
                    "type": "image_url",
                    "image_url":{
                        "url":f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ]
    response=client.chat.completions.create(
        model="gpt-4o",
        messages=messages
)
    return response.choices[0].message.content

In [4]:
q = image_quiz("C:/llm01/gpt/images/busan_dive.jpg")
print(q)

Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?

1. 많은 사람들이 테이블에 앉아 노트북을 사용하고 있습니다.
2. 천장에 조명이 설치되어 있습니다.
3. 'DIVE 2024 IN BUSAN'이라는 문구가 보입니다.
4. 모든 사람들이 서서 이야기를 나누고 있습니다.

정답: 4. 모든 사람들이 서서 이야기를 나누고 있지 않고, 대부분 앉아 있습니다.


In [ ]:
txt='' #모든 문제 내용을 하나의 문자열로 누적 저장하기 위한 변수
no=1 #문제 번화

#모든 이미지 가져오기, g:각 이미지 파일의 전체 경로
for g in glob('C:/llm01/gpt/images/*.jpg'):
    try:
        q=image_quiz(g)
    except Exception as e:
        print(e)
        continue

#문제 1,2 ... 나누기
    divider = f'## 문제 {no}\n\n'
    print(divider)

    txt += divider
    filename=os.path.basename(g) #이미지 파일 경로에서 파일명만 분리
    txt += f'![image]({filename})\n\n'

    print(q)
    txt += q + '\n\n----------------\n\n'

    #퀴즈 내용 마크다운 파일로 저장
    with open('C:/llm01/gpt/images/image_quiz.md','w',encoding='utf-8') as f: 
        f.write(txt)

    no+=1

## 문제 1


    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
    1. 사람들이 컴퓨터를 사용하며 앉아 있습니다.
    2. 천장에 조명이 설치되어 있습니다.
    3. "DIVE 2024 IN BUSAN"이라는 문구가 보입니다.
    4. 이미지에는 차가 두 대 보입니다.

    정답: 4. 이미지에는 차가 보이지 않습니다.
## 문제 2



Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 노란색 조형물이 건물 앞에 있습니다.
2. 'Local Stitch'라는 글자가 보입니다.
3. 조형물은 파란색입니다.
4. 배경 건물은 여러 층으로 이루어져 있습니다.

정답: 3. 조형물은 노란색입니다, 파란색이 아닙니다.
## 문제 3


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 카페 안에 의자와 테이블이 배치되어 있습니다.
2. 카운터 뒤 벽은 노란색으로 칠해져 있습니다.
3. 카운터에는 많은 사람들이 줄 서 있습니다.
4. 넓은 창문을 통해 자연광이 들어오고 있습니다.

정답: 3. 카운터에는 많은 사람들이 줄 서 있는 것이 아닙니다.
## 문제 4


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?

1. 건물의 첫 번째 층에는 커피숍이 있습니다.
2. 건물은 벽돌로 지어져 있습니다.
3. 건물 옥상에는 사람들이 보입니다.
4. 건물 앞에는 주황색 콘이 놓여 있습니다.

정답: 3. 건물 옥상에는 사람들이 보이지 않습니다.
## 문제 5


---

Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 진열대에 다양한 종류의 빵이 놓여 있습니다.
2. 직원들은 흰색 모자를 쓰고 있습니다.
3. 모든 빵에는 가격표가 부착되어 있습니다.
4. 에어컨은 작동 중입니다.

정답: 4. 에어컨은 작동 중인 상태가 아닙니다. (이미지만으로 알 수 없습니다.)
## 문제 6


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?

1. 이미지에는 '뉴트